In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

# Dummy data
data = {
    "Notes": ["Cannot login", "Email not working", "WiFi down", "Computer crashes", "VPN fails", "Outlook error", "Slow internet", "Account locked"],
    "close_notes": ["User reset password", "Checked SMTP", "Router restarted", "Hardware issue", "Reconfigured VPN", "Updated settings", "ISP problem", "Manually unlocked"],
    "Incident": ["User issue", "Email failure", "Connectivity", "Hardware", "VPN problem", "Email failure", "Connectivity", "User issue"],
    "Issue Theme": ["Login Issue", "Email Problem", "Network Error", "Hardware Failure", "VPN Issue", "Email Problem", "Network Error", "Login Issue"]
}

label_descriptions = {
    "Login Issue": "Problems related to signing in or authentication",
    "Email Problem": "Issues with sending, receiving, or configuring emails",
    "Network Error": "Connectivity issues such as internet outage or network slowness",
    "Hardware Failure": "Physical device problems like system crashes or device not powering on",
    "VPN Issue": "VPN-related configuration or connection issues",
    "Software Bug": "Issues caused by faulty software behavior",
    "Security Alert": "Security or breach-related issues",
    "Performance Problem": "Slow or lagging system responses"
}

df = pd.DataFrame(data)

# Combine text
df['combined'] = df['Notes'] + " " + df['close_notes']
df['label'] = df['Issue Theme']

# Encode labels
labels = df['label'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}
df['label_idx'] = df['label'].map(label_to_index)

# Prepare class descriptions (context)
context_embeddings = []
for label in df['label']:
    description = label_descriptions.get(label, "")
    ctx_emb = get_embedding(description)
    context_embeddings.append(ctx_emb[0])  # Assuming shape [1, dim]
df['context_emb'] = context_embeddings

# Get input embeddings
df['input_emb'] = df['combined'].apply(lambda x: get_embedding(x)[0])  # Assuming shape [1, dim]

# Train/test split
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42, stratify=df['label_idx'])

# PyTorch dataset
class IncidentDataset(Dataset):
    def __init__(self, df):
        self.X = df['input_emb'].tolist()
        self.C = df['context_emb'].tolist()
        self.y = df['label_idx'].tolist()

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float32)
        ctx = torch.tensor(self.C[idx], dtype=torch.float32)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return x, ctx, y

train_set = IncidentDataset(train_df)
test_set = IncidentDataset(test_df)

train_loader = DataLoader(train_set, batch_size=2, shuffle=True)
test_loader = DataLoader(test_set, batch_size=2)

# Simple model
class IncidentClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim * 2, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x, ctx):
        combined = torch.cat([x, ctx], dim=1)
        return self.fc(combined)

# Initialize model
input_dim = len(df['input_emb'].iloc[0])
num_classes = len(label_to_index)
model = IncidentClassifier(input_dim=input_dim, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(20):
    model.train()
    total_loss = 0
    for x, ctx, y in train_loader:
        optimizer.zero_grad()
        outputs = model(x, ctx)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Evaluation
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for x, ctx, y in test_loader:
        outputs = model(x, ctx)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(y.tolist())

# Report
index_to_label = {v: k for k, v in label_to_index.items()}
pred_names = [index_to_label[p] for p in all_preds]
true_names = [index_to_label[t] for t in all_labels]

print("\nDetailed Classification Report:\n")
print(classification_report(true_names, pred_names, digits=4))
